In [1]:
#!pip install pytube

# Download YouTube Video Locally 

In [2]:
from pytube import YouTube 

def download_video_from_youtube(link, file_name):
    
    yt = YouTube(link)   
    stream = yt.streams.first()
    file_path = stream.download(filename=file_name)  
    
    print(f"The file is saved at : {file_path}")
    
    return file_path

In [3]:
# link of the video to be downloaded (Update this link)
link = "https://www.youtube.com/watch?v=k9f1TTiqWr4" 

# Name of the saved file (Update with any name of your choice)
file_name = "DataLakeinAmazonS3Kumar.mp4"
final_file_name = "DataLakeinAmazonS3Kumar"

In [4]:
file_path = download_video_from_youtube(link, file_name)

The file is saved at : /home/ec2-user/SageMaker/AI_Demos/DataLakeinAmazonS3Kumarmp4.mp4


# Upload the file to S3

In [5]:
import boto3
session = boto3.Session()

def upload_file(file_name, bucket, object_name=None):
    
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    
    return True

In [6]:
# Update your bucket name 
bucket= 'my-ai-bucket-suman'

upload_file(file_path, bucket, object_name=file_name)

True

# Amazon Transcribe

In [7]:
client = session.client('transcribe')

In [8]:
job_name = "myjob1"
job_uri = f"https://{bucket}.s3.amazonaws.com/{file_name}"

In [9]:
client.start_transcription_job(TranscriptionJobName=job_name, 
                               Media={'MediaFileUri': job_uri}, 
                               MediaFormat='mp4', LanguageCode='en-US'
                              )

{'TranscriptionJob': {'TranscriptionJobName': 'myjob1',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'en-US',
  'MediaFormat': 'mp4',
  'Media': {'MediaFileUri': 'https://my-ai-bucket-suman.s3.amazonaws.com/DataLakeinAmazonS3Kumar.mp4'},
  'StartTime': datetime.datetime(2021, 7, 18, 13, 25, 0, 972000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2021, 7, 18, 13, 25, 0, 952000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': 'f4587330-f455-4f11-a52f-c9a27cf50294',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 18 Jul 2021 13:25:00 GMT',
   'x-amzn-requestid': 'f4587330-f455-4f11-a52f-c9a27cf50294',
   'content-length': '296',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [10]:
import time
import urllib.request

while True:
    status = client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(10)
    
print(f'TranscriptionJobStatus : {status["TranscriptionJob"]["TranscriptionJobStatus"]}')

Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
TranscriptionJobStatus : COMPLETED


In [11]:
response = client.get_transcription_job(TranscriptionJobName=job_name)
url = response["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]
output_file_name = urllib.request.urlretrieve(url, f"{final_file_name}.json")[0]

In [12]:
import json

with open(output_file_name) as json_file:
    data = json.load(json_file)

final_text = data["results"]["transcripts"][0]["transcript"]

with open(f"{final_file_name}.txt", "w") as text_file:
    text_file.write(final_text)
    

In [13]:
print(f"Final transcribed file is saved in the present working directory : {final_file_name}.txt")

Final transcribed file is saved in the present working directory : DataLakeinAmazonS3Kumar.txt


# Delete the job

In [14]:
client.delete_transcription_job(TranscriptionJobName=job_name)

{'ResponseMetadata': {'RequestId': '30e9299f-e711-485c-9164-be092388ae5e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 18 Jul 2021 13:31:23 GMT',
   'x-amzn-requestid': '30e9299f-e711-485c-9164-be092388ae5e',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}